In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
%matplotlib inline
from annoy import AnnoyIndex


imsize=224
### resnet stuff
extractor_resnet = models.resnet101(pretrained=True)
extractor_resnet.eval()
num_ftrs = 1000

loader = transforms.Compose([transforms.Resize((imsize,imsize)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    #image_loader_1 = PIL.Image.fromarray(image_name)
    image_loader_1 = PIL.Image.open(image_name).convert("RGB")
    image_loader_1 = loader(image_loader_1).float()
    image_loader_1 = Variable(image_loader_1, requires_grad=True)
    image_loader_1 = image_loader_1.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    y_pred = extractor_resnet(image_loader_1)
    return y_pred.tolist()[0]#.cuda()  #assumes that you're using GPU

def showImagesHorizontally(annoy_dict, base_img, dir_, list_of_files,index):
    fig = figure(figsize=(160,90))
    number_of_files = len(list_of_files)
    list_of_files = [base_img] + list_of_files
    first_bool = 0 
    for i in range(number_of_files):
        if first_bool ==0:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(list_of_files[i])
            first_bool = 1
            
        else:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(dir_+annoy_dict[str(list_of_files[i])])
        
        #image = imread(dir_l+annoy_dict[str(list_of_files[i])])
        imshow(image,aspect='equal')
        axis('off')
    fig.savefig('D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/examples/{}.png'.format(index))
    fig.clf()

    
annoy_index = AnnoyIndex(1000,metric='euclidean')
annoy_index.load('annoy_euclidean_fgo.ann')


True

In [2]:
dat = pd.read_csv('mapping_with_annoy.csv')

dict_fgo_annoy = {}
for index,row in dat.iterrows():
    dict_fgo_annoy[str(row['annoy_index'])] = str(row['original_image']) #switch this out

    
dat_second = pd.read_csv('test_set_head_base_mapping.csv')

dat_second_dict = {}
for index,row in dat_second.iterrows():
    img_name = row['head_name']
    feature_vec = image_loader(row['dir_loc']+img_name)
    test_list = annoy_index.get_nns_by_vector(feature_vec, 5)
    
    base_image_path = 'F:/fgo_multi_label/test/'+row['original_image']
    dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/all_images/'

    showImagesHorizontally(dict_fgo_annoy,base_image_path,dir_loc,test_list,index=index)      

#dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/'

#for test_image in test_images:
    
#    img_name = test_image
#    feature_vec = image_loader(dir_loc+'heads/'+img_name)
#    test_list = annoy_index.get_nns_by_vector(feature_vec, 10)

    #dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/heads/'



#    showImagesHorizontally(dict_fgo_annoy,dir_loc,test_list)      

d:\users\micha\appdata\local\programs\python\python35\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
